In [2]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse

In [3]:
country_cd = []
code = []
req = requests.get('https://www.mobileaction.co/top-apps/travel-6003/ios/us')
all = BeautifulSoup(req.content, 'html.parser')
v = all.find_all('i', class_='flag-icon')
for b in v:
    pro = str(b.parent.get_text()).replace(' ', '')
    country_cd.append(pro.strip('\n'))

In [148]:
iso = pd.read_csv('/content/drive/MyDrive/Travel Industry Analysis/datasets/iso_code.csv')
c_code = []
for country in country_cd:
    c_code.append([f for f in iso.loc[iso['Name'] == re.sub(r"(\w)([A-Z])", r"\1 \2", country)].Code.values])

out = [x for x in c_code]
f_code = pd.DataFrame(out, columns=['Code']).dropna()

vc = [d for d in f_code.Code]
df = pd.DataFrame(columns=['CountryISO', 'AppName', 'AppID', 'appURL', 'Ratings', 'ratingScore', 
                           'Developer', 'websiteURL', 'Category', 'Downloads', 'downloadSize', 'currentVersion', 'Device', 'Usage', 'Platform'])
len(vc)

80

In [167]:
for c in vc[78:]:
  res = requests.get(f'https://www.mobileaction.co/top-apps/travel-6003/ios/{c}')
  soup = BeautifulSoup(res.content, 'html.parser')
  free = soup.select('.free-col')
  for x in free:
    cvv = BeautifulSoup(str(x), 'html.parser')
    pro = str(cvv.find('a', class_='app-name').get_text()).replace(' ', '')
    l = urlparse(f"https://www.mobileaction.co{x.select_one('.app-name')['href']}")
    id = l.path.rsplit('/', 1)[-1]
    dev = str(cvv.find('span', class_='ma-text-overflow block').get_text()).replace(' ', '')
    appurl = f"https://apps.apple.com/us/app/id{id}"

    rev = requests.get(f"https://www.mobileaction.co{x.select_one('.app-name')['href']}")
    m_soup = BeautifulSoup(rev.content, 'html.parser')
    try:
      dl = m_soup.select_one('.ma-dr-amount').text.replace(' ', '').strip('\n')
    except:
      dl = None
    
    df = df.append({'CountryISO': c,
                    'AppName': pro.strip('\n'),
                    'AppID': id,
                    'appURL': appurl,
                    'Developer': dev.strip('\n'),
                    'Downloads': dl,
                    'Usage': 'Free',
                    'Platform': 'IOS'}, ignore_index=True)

In [170]:
df.drop_duplicates(subset='AppID', inplace=True)
df.reset_index(inplace=True)
df

,index,CountryISO,AppName,AppID,appURL,Ratings,ratingScore,Developer,websiteURL,Category,Downloads,downloadSize,currentVersion,Device,Usage,Platform
0,0,US,Upside-Cashbackongas&food,1099997174,https://apps.apple.com/us/app/id1099997174,NaN,NaN,UpsideServicesInc,NaN,NaN,2M,NaN,NaN,NaN,Free,IOS
1,1,US,Uber-Requestaride,368677368,https://apps.apple.com/us/app/id368677368,NaN,NaN,"UberTechnologies,Inc.",NaN,NaN,1M,NaN,NaN,NaN,Free,IOS
2,2,US,VrboVacationRentals,1245772818,https://apps.apple.com/us/app/id1245772818,NaN,NaN,"HomeAway.com,Inc.",NaN,NaN,1M,NaN,NaN,NaN,Free,IOS
3,3,US,Booking.com:Hotels&Travel,367003839,https://apps.apple.com/us/app/id367003839,NaN,NaN,Booking.com,NaN,NaN,760K,NaN,NaN,NaN,Free,IOS
4,4,US,Airbnb,401626263,https://apps.apple.com/us/app/id401626263,NaN,NaN,"Airbnb,Inc.",NaN,NaN,982K,NaN,NaN,NaN,Free,IOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,7742,TN,DeltaAirlinesAirSonar,1102430878,https://apps.apple.com/us/app/id1102430878,NaN,NaN,FikretUrgan,NaN,NaN,,NaN,NaN,NaN,Free,IOS
2579,7744,TN,JipiTaxi,1449425062,https://apps.apple.com/us/app/id1449425062,NaN,NaN,DoneSantana,NaN,NaN,,NaN,NaN,NaN,Free,IOS
2580,7745,TN,HermesTunisia,1631832596,https://apps.apple.com/us/app/id1631832596,NaN,NaN,datasoundsrl,NaN,NaN,,NaN,NaN,NaN,Free,IOS
2581,7746,TN,SwissInternationalHotels,1629690277,https://apps.apple.com/us/app/id1629690277,NaN,NaN,SwissInternational,NaN,NaN,,NaN,NaN,NaN,Free,IOS


In [ ]:
for ind, app in df[2086:].iterrows():
  res = requests.get(app.appURL)
  print(app.appURL)
  if res.status_code != 404:
    soup = BeautifulSoup(res.content, 'html.parser')
    try:
      rating_score = soup.select_one('.we-customer-ratings__averages__display').text
      ratings = soup.select_one('.we-customer-ratings__count').text.split(' ')[0]
      c_ver = soup.select_one('.whats-new__latest__version').text.split(' ')[1]
      website = soup.select_one('.link.icon.icon-after.icon-external')['href']
    except:
      rating_score = None
      ratings = None
      c_ver = None
      website = None
    cate = soup.find("a", {'href': 'https://itunes.apple.com/us/genre/id6003'}).get_text().strip('  \n')
    d_size = soup.find_all('dd', class_='information-list__item__definition')[1].get_text()
    device = soup.find_all('dd', class_='information-list__item__definition')[3].get_text().strip("  \n")

    df.loc[df.AppID == app.AppID, 'Ratings'] = ratings
    df.loc[df.AppID == app.AppID, 'ratingScore'] = rating_score
    df.loc[df.AppID == app.AppID, 'Category'] = cate
    df.loc[df.AppID == app.AppID, 'downloadSize'] = d_size
    df.loc[df.AppID == app.AppID, 'Device'] = device
    df.loc[df.AppID == app.AppID, 'websiteURL'] = website
    df.loc[df.AppID == app.AppID, 'currentVersion'] = c_ver
    print(ind)
  else:
    continue


In [1]:
df

NameError: ignored